In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('whitegrid')
import re

In [2]:
df=pd.read_csv('Preliminary_database4.csv', index_col=0)

In [3]:
df

Title CELEX number  \
0    Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
2    Judgment of the Court (Fifth Chamber) of 23 Ma...  62017CJ0720   
3    Judgment of the General Court (Third Chamber) ...  62017TJ0107   
4    Judgment of the General Court (Fourth Chamber)...  62015TJ0604   
..                                                 ...          ...   
593  Judgment of the General Court (Eighth Chamber)...  62005TJ0011   
594  Judgment of the Court (Fourth Chamber) of 6 Ma...  62008CJ0145   
595  Judgment of the Court (Fourth Chamber) of 18 M...  62008CJ0317   
596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   

        ECLI identifier                                  Type of procedure  \
0    ECLI:EU:C:2019:757  Reference for a preliminary ruling - inadmissi...   
1    ECLI:EU:C:2019:634  Action for annulment, Appeal - inadmissible, A...   
2    ECLI:EU:C:2019:448                 Reference for a preliminary ruling   
3    ECLI:EU:T:2019:353                     Action for damages - unfounded   
4    ECLI:EU:T:2019:348                   Action for annulment - unfounded   
..                  ...                                                ...   
593  ECLI:EU:T:2010:201  Action for annulment - unfounded, Appeal again...   
594  ECLI:EU:C:2010:247                 Reference for a preliminary ruling   
595  ECLI:EU:C:2010:146                 Reference for a preliminary ruling   
596   ECLI:EU:T:2010:54  Action for annulment - inadmissible, Action fo...   
597   ECLI:EU:T:2010:55  Action for annulment - inadmissible, Action fo...   

         origin             Author Applicant/Appellant  \
0         Spain  domestic referral                 NaN   
1       Germany  domestic referral              Person   
2       Austria  domestic referral                 NaN   
3       Germany      General Court              Person   
4       Belgium      General Court              Person   
..          ...                ...                 ...   
593     Austria      General Court              Person   
594      Greece  domestic referral                 NaN   
595       Italy  domestic referral                 NaN   
596  Luxembourg      General Court              Person   
597      Greece      General Court              Person   

                                           parties  \
0                                              NaN   
1                 Commission, Institutions, Person   
2                                              NaN   
3              Institutions, European Central Bank   
4                         Institutions, Commission   
..                                             ...   
593                       Commission, Institutions   
594                                            NaN   
595                                            NaN   
596     Council, Institutions, European Parliament   
597  Institutions, European Maritime Safety Agency   

                                                  Text        date  year  
0      JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019  
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019  
2      JUDGMENT OF THE COURT (Fifth Chamber)  May (...  2019-05-23  2019  
3       JUDGMENT OF THE GENERAL COURT (Third Chambe...  2019-05-23  2019  
4      JUDGMENT OF THE GENERAL COURT (Fourth Chambe...  2019-05-22  2019  
..                                                 ...         ...   ...  
593    Judgment of the General Court (Eighth Chambe...  2010-05-19  2010  
594    Joined Cases C-145/and C-149/Club Hotel Lout...  2010-05-06  2010  
595    Joined Cases C-317/to C-320/Rosalba Alassini...  2010-03-18  2010  
596    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010  
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010  

[598 rows x 

In [4]:
## as we can see, this is comma separated, so the first thing we want to do is to split into a list
df['Type of procedure'].value_counts()


Reference for a preliminary ruling                                                                            132
Action for annulment - unfounded                                                                               72
Action for annulment - successful                                                                              24
Action for annulment, Appeal - unfounded                                                                       15
Action for damages - unfounded, Action for annulment - unfounded, Action for annulment - inadmissible          15
                                                                                                             ... 
Action for annulment, Appeal - successful, Action for annulment - unfounded                                     1
Action for damages - unfounded, Action for annulment - unfounded, Action for failure to act - inadmissible      1
Action for annulment - unfounded, Action for annulment - successful, Appeal against pena

In [5]:
# since it is a simple function that we will only use once, a lambda looks nice
df['Type of procedure'] = df['Type of procedure'].apply(lambda x: x.split(","))

In [6]:
df3 = df.explode('Type of procedure')
## now we can see what is going on, it is a horrible mess
df3['Type of procedure'].value_counts()

Reference for a preliminary ruling                  138
Action for annulment - unfounded                    112
 Action for annulment - unfounded                    82
 Appeal - unfounded                                  69
 Action for annulment                                68
                                                   ... 
 Action for failure to act - unfounded                1
 Application for measures of inquiry - unfounded      1
 Arbitration clause - successful                      1
 Arbitration clause - unfounded                       1
 Action for failure to act - inadmissible             1
Name: Type of procedure, Length: 61, dtype: int64

>a strategy here is to focus on the regularities, for example unfounded, succesful etc, here you can follow the logic of the trimming

In [7]:
def type_standarizer2(a_string):
    if "unfounded" in a_string:
        return "unfounded"
    elif "successful" in a_string:
        return "succesful"
    elif "inadmissible" in a_string:
        return "inadmissible"
    elif "unnecessary" in a_string:
        return "unnecessary"
    elif "Staff cases" in a_string:
        return "staff cases"
    elif "Action for annulment" in a_string:
        return "action for annulment"
    elif "Action for damages" in a_string:
        return "action for damages"
    else:
        return a_string    
    

In [8]:
df3['outcome'] = df3['Type of procedure'].apply(type_standarizer2)
df3.outcome.value_counts()

unfounded                                   430
succesful                                   210
Reference for a preliminary ruling          138
inadmissible                                128
action for annulment                        122
 Appeal against penalty                      36
action for damages                           21
unnecessary                                  18
Appeal against penalty                       12
staff cases                                  12
Preliminary reference - urgent procedure     10
 Reference for a preliminary ruling          10
 Appeal                                       7
Judgment delivered after back reference       5
 Judgment delivered after back reference      2
Appeal                                        1
Arbitration clause                            1
Name: outcome, dtype: int64

In [9]:
#it creates duplicates or even triplicates of cases!
df3

Title CELEX number  \
0    Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
0    Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
..                                                 ...          ...   
596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   

        ECLI identifier                                  Type of procedure  \
0    ECLI:EU:C:2019:757  Reference for a preliminary ruling - inadmissible   
0    ECLI:EU:C:2019:757                 Reference for a preliminary ruling   
1    ECLI:EU:C:2019:634                               Action for annulment   
1    ECLI:EU:C:2019:634                              Appeal - inadmissible   
1    ECLI:EU:C:2019:634                                 Appeal - unfounded   
..                  ...                                                ...   
596   ECLI:EU:T:2010:54                Action for annulment - inadmissible   
596   ECLI:EU:T:2010:54                     Action for damages - unfounded   
597   ECLI:EU:T:2010:55                Action for annulment - inadmissible   
597   ECLI:EU:T:2010:55                  Action for annulment - successful   
597   ECLI:EU:T:2010:55                   Action for annulment - unfounded   

         origin             Author Applicant/Appellant  \
0         Spain  domestic referral                 NaN   
0         Spain  domestic referral                 NaN   
1       Germany  domestic referral              Person   
1       Germany  domestic referral              Person   
1       Germany  domestic referral              Person   
..          ...                ...                 ...   
596  Luxembourg      General Court              Person   
596  Luxembourg      General Court              Person   
597      Greece      General Court              Person   
597      Greece      General Court              Person   
597      Greece      General Court              Person   

                                           parties  \
0                                              NaN   
0                                              NaN   
1                 Commission, Institutions, Person   
1                 Commission, Institutions, Person   
1                 Commission, Institutions, Person   
..                                             ...   
596     Council, Institutions, European Parliament   
596     Council, Institutions, European Parliament   
597  Institutions, European Maritime Safety Agency   
597  Institutions, European Maritime Safety Agency   
597  Institutions, European Maritime Safety Agency   

                                                  Text        date  year  \
0      JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
0      JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
..                                                 ...         ...   ...   
596    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
596    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   



In [10]:
#I figured that the only values that are valuable are successful, unfounded and inadmissible
df3['outcome'].replace(['Reference for a preliminary ruling', 'action for annulment', ' Appeal against penalty',
                       'action for damages', 'unnecessary', 'staff cases', 'Appeal against penalty', 
                        ' Reference for a preliminary ruling', 'Preliminary reference - urgent procedure',
                       ' Appeal', 'Judgment delivered after back reference', ' Judgment delivered after back reference', 
                       'Appeal', 'Arbitration clause'], np.nan, inplace=True)

In [11]:
df3.head()

Title CELEX number  \
0  Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
0  Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1  Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
1  Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
1  Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   

      ECLI identifier                                  Type of procedure  \
0  ECLI:EU:C:2019:757  Reference for a preliminary ruling - inadmissible   
0  ECLI:EU:C:2019:757                 Reference for a preliminary ruling   
1  ECLI:EU:C:2019:634                               Action for annulment   
1  ECLI:EU:C:2019:634                              Appeal - inadmissible   
1  ECLI:EU:C:2019:634                                 Appeal - unfounded   

    origin             Author Applicant/Appellant  \
0    Spain  domestic referral                 NaN   
0    Spain  domestic referral                 NaN   
1  Germany  domestic referral              Person   
1  Germany  domestic referral              Person   
1  Germany  domestic referral              Person   

                            parties  \
0                               NaN   
0                               NaN   
1  Commission, Institutions, Person   
1  Commission, Institutions, Person   
1  Commission, Institutions, Person   

                                                Text        date  year  \
0    JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
0    JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1    ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
1    ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
1    ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   

        outcome  
0  inadmissible  
0           NaN  
1           NaN  
1  inadmissible  
1     unfounded

In [12]:
df3.columns.to_list()

['Title',
 'CELEX number',
 'ECLI identifier',
 'Type of procedure',
 'origin',
 'Author',
 'Applicant/Appellant',
 'parties',
 'Text',
 'date',
 'year',
 'outcome']

In [13]:
df4=df3[df3['outcome'].isnull()]

In [14]:
#creates df with NaN in outcome and drops duplicates
df4=df4.drop_duplicates(subset=['CELEX number'])
df4

Title CELEX number  \
0    Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
2    Judgment of the Court (Fifth Chamber) of 23 Ma...  62017CJ0720   
6    Judgment of the Court (First Chamber) of 8 May...  62017CJ0396   
7    Judgment of the Court (First Chamber) of 8 May...  62017CJ0024   
..                                                 ...          ...   
585  JUDGMENT OF THE GENERAL COURT (Appeal Chamber)...  62008TJ0017   
588  Judgment of the Court (Second Chamber) of 1 Ju...  62009CJ0035   
589  Judgment of the Court (Grand Chamber) of 29 Ju...  62008CJ0028   
594  Judgment of the Court (Fourth Chamber) of 6 Ma...  62008CJ0145   
595  Judgment of the Court (Fourth Chamber) of 18 M...  62008CJ0317   

        ECLI identifier                    Type of procedure          origin  \
0    ECLI:EU:C:2019:757   Reference for a preliminary ruling           Spain   
1    ECLI:EU:C:2019:634                 Action for annulment         Germany   
2    ECLI:EU:C:2019:448   Reference for a preliminary ruling         Austria   
6    ECLI:EU:C:2019:375   Reference for a preliminary ruling         Austria   
7    ECLI:EU:C:2019:373   Reference for a preliminary ruling         Austria   
..                  ...                                  ...             ...   
585  ECLI:EU:T:2010:374                          Staff cases    not possible   
588  ECLI:EU:C:2010:393   Reference for a preliminary ruling           Italy   
589  ECLI:EU:C:2010:378                 Action for annulment  United Kingdom   
594  ECLI:EU:C:2010:247   Reference for a preliminary ruling          Greece   
595  ECLI:EU:C:2010:146   Reference for a preliminary ruling           Italy   

                Author       Applicant/Appellant  \
0    domestic referral                       NaN   
1    domestic referral                    Person   
2    domestic referral                       NaN   
6    domestic referral                       NaN   
7    domestic referral                       NaN   
..                 ...                       ...   
585      General Court                  Official   
588  domestic referral                       NaN   
589  domestic referral  Commission, Institutions   
594  domestic referral                       NaN   
595  domestic referral                       NaN   

                              parties  \
0                                 NaN   
1    Commission, Institutions, Person   
2                                 NaN   
6                                 NaN   
7                                 NaN   
..                                ...   
585          Commission, Institutions   
588                               NaN   
589                            Person   
594                               NaN   
595                               NaN   

                                                  Text        date  year  \
0      JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
2      JUDGMENT OF THE COURT (Fifth Chamber)  May (...  2019-05-23  2019   
6      JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
7      JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
..                                                 ...         ...   ...   
585    ARRÊT DU TRIBUNAL (chambre des pourvois)  se...  2010-09-09  2010   
588    Case C-35/Ministero dell'Economia e delle Fi...  2010-07-01  2010   
589    Case C-28/P  European Commission  v  The Bav...  2010-06-29  2010   
594    Joined Cases C-145/and C-149/Club Hotel Lout...  2010-05-06  2010   
595    Joined Cases C-317/to C-320/Rosalba Alassini...  2010-03-18  2010   

    outcome  
0       NaN  
1       NaN  
2       NaN  
6       NaN  
7       NaN  
..      ...  
585     NaN  
588     NaN  
589     NaN  
594     NaN  
595     NaN  

[282 rows x 12 columns]

In [15]:
#drops nan in outcome
df5=df3.dropna(subset=['outcome'])

In [16]:
df5

Title CELEX number  \
0    Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
1    Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
3    Judgment of the General Court (Third Chamber) ...  62017TJ0107   
4    Judgment of the General Court (Fourth Chamber)...  62015TJ0604   
..                                                 ...          ...   
596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   

        ECLI identifier                                  Type of procedure  \
0    ECLI:EU:C:2019:757  Reference for a preliminary ruling - inadmissible   
1    ECLI:EU:C:2019:634                              Appeal - inadmissible   
1    ECLI:EU:C:2019:634                                 Appeal - unfounded   
3    ECLI:EU:T:2019:353                     Action for damages - unfounded   
4    ECLI:EU:T:2019:348                   Action for annulment - unfounded   
..                  ...                                                ...   
596   ECLI:EU:T:2010:54                Action for annulment - inadmissible   
596   ECLI:EU:T:2010:54                     Action for damages - unfounded   
597   ECLI:EU:T:2010:55                Action for annulment - inadmissible   
597   ECLI:EU:T:2010:55                  Action for annulment - successful   
597   ECLI:EU:T:2010:55                   Action for annulment - unfounded   

         origin             Author Applicant/Appellant  \
0         Spain  domestic referral                 NaN   
1       Germany  domestic referral              Person   
1       Germany  domestic referral              Person   
3       Germany      General Court              Person   
4       Belgium      General Court              Person   
..          ...                ...                 ...   
596  Luxembourg      General Court              Person   
596  Luxembourg      General Court              Person   
597      Greece      General Court              Person   
597      Greece      General Court              Person   
597      Greece      General Court              Person   

                                           parties  \
0                                              NaN   
1                 Commission, Institutions, Person   
1                 Commission, Institutions, Person   
3              Institutions, European Central Bank   
4                         Institutions, Commission   
..                                             ...   
596     Council, Institutions, European Parliament   
596     Council, Institutions, European Parliament   
597  Institutions, European Maritime Safety Agency   
597  Institutions, European Maritime Safety Agency   
597  Institutions, European Maritime Safety Agency   

                                                  Text        date  year  \
0      JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
1      ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
3       JUDGMENT OF THE GENERAL COURT (Third Chambe...  2019-05-23  2019   
4      JUDGMENT OF THE GENERAL COURT (Fourth Chambe...  2019-05-22  2019   
..                                                 ...         ...   ...   
596    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
596    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
597    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   



In [38]:
#I have removed with that duplicates of NaN, but the rest of duplicates are still there
df6=pd.concat([df4, df5])
df6=df6.reset_index()
df6

index                                              Title CELEX number  \
0         0  Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1         1  Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
2         2  Judgment of the Court (Fifth Chamber) of 23 Ma...  62017CJ0720   
3         6  Judgment of the Court (First Chamber) of 8 May...  62017CJ0396   
4         7  Judgment of the Court (First Chamber) of 8 May...  62017CJ0024   
...     ...                                                ...          ...   
1045    596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
1046    596  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
1047    597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
1048    597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
1049    597  Judgment of the General Court (Third Chamber) ...  62005TJ0070   

         ECLI identifier                    Type of procedure      origin  \
0     ECLI:EU:C:2019:757   Reference for a preliminary ruling       Spain   
1     ECLI:EU:C:2019:634                 Action for annulment     Germany   
2     ECLI:EU:C:2019:448   Reference for a preliminary ruling     Austria   
3     ECLI:EU:C:2019:375   Reference for a preliminary ruling     Austria   
4     ECLI:EU:C:2019:373   Reference for a preliminary ruling     Austria   
...                  ...                                  ...         ...   
1045   ECLI:EU:T:2010:54  Action for annulment - inadmissible  Luxembourg   
1046   ECLI:EU:T:2010:54       Action for damages - unfounded  Luxembourg   
1047   ECLI:EU:T:2010:55  Action for annulment - inadmissible      Greece   
1048   ECLI:EU:T:2010:55    Action for annulment - successful      Greece   
1049   ECLI:EU:T:2010:55     Action for annulment - unfounded      Greece   

                 Author Applicant/Appellant  \
0     domestic referral                 NaN   
1     domestic referral              Person   
2     domestic referral                 NaN   
3     domestic referral                 NaN   
4     domestic referral                 NaN   
...                 ...                 ...   
1045      General Court              Person   
1046      General Court              Person   
1047      General Court              Person   
1048      General Court              Person   
1049      General Court              Person   

                                            parties  \
0                                               NaN   
1                  Commission, Institutions, Person   
2                                               NaN   
3                                               NaN   
4                                               NaN   
...                                             ...   
1045     Council, Institutions, European Parliament   
1046     Council, Institutions, European Parliament   
1047  Institutions, European Maritime Safety Agency   
1048  Institutions, European Maritime Safety Agency   
1049  Institutions, European Maritime Safety Agency   

                                                   Text        date  year  \
0       JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1       ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
2       JUDGMENT OF THE COURT (Fifth Chamber)  May (...  2019-05-23  2019   
3       JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
4       JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
...                                                 ...         ...   ...   
1045    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1046    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1047    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1048    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1049    JUDGMENT OF THE GENERAL COURT (Third Chamber...  20

In [39]:
df6=df6.drop('index', 1)
df6

Title CELEX number  \
0     Judgment of the Court (Sixth Chamber) of 18 Se...  62018CJ0366   
1     Judgment of the Court (Fifth Chamber) of 29 Ju...  62017CJ0654   
2     Judgment of the Court (Fifth Chamber) of 23 Ma...  62017CJ0720   
3     Judgment of the Court (First Chamber) of 8 May...  62017CJ0396   
4     Judgment of the Court (First Chamber) of 8 May...  62017CJ0024   
...                                                 ...          ...   
1045  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
1046  Judgment of the General Court (Third Chamber) ...  62004TJ0016   
1047  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
1048  Judgment of the General Court (Third Chamber) ...  62005TJ0070   
1049  Judgment of the General Court (Third Chamber) ...  62005TJ0070   

         ECLI identifier                    Type of procedure      origin  \
0     ECLI:EU:C:2019:757   Reference for a preliminary ruling       Spain   
1     ECLI:EU:C:2019:634                 Action for annulment     Germany   
2     ECLI:EU:C:2019:448   Reference for a preliminary ruling     Austria   
3     ECLI:EU:C:2019:375   Reference for a preliminary ruling     Austria   
4     ECLI:EU:C:2019:373   Reference for a preliminary ruling     Austria   
...                  ...                                  ...         ...   
1045   ECLI:EU:T:2010:54  Action for annulment - inadmissible  Luxembourg   
1046   ECLI:EU:T:2010:54       Action for damages - unfounded  Luxembourg   
1047   ECLI:EU:T:2010:55  Action for annulment - inadmissible      Greece   
1048   ECLI:EU:T:2010:55    Action for annulment - successful      Greece   
1049   ECLI:EU:T:2010:55     Action for annulment - unfounded      Greece   

                 Author Applicant/Appellant  \
0     domestic referral                 NaN   
1     domestic referral              Person   
2     domestic referral                 NaN   
3     domestic referral                 NaN   
4     domestic referral                 NaN   
...                 ...                 ...   
1045      General Court              Person   
1046      General Court              Person   
1047      General Court              Person   
1048      General Court              Person   
1049      General Court              Person   

                                            parties  \
0                                               NaN   
1                  Commission, Institutions, Person   
2                                               NaN   
3                                               NaN   
4                                               NaN   
...                                             ...   
1045     Council, Institutions, European Parliament   
1046     Council, Institutions, European Parliament   
1047  Institutions, European Maritime Safety Agency   
1048  Institutions, European Maritime Safety Agency   
1049  Institutions, European Maritime Safety Agency   

                                                   Text        date  year  \
0       JUDGMENT OF THE COURT (Sixth Chamber)  Septe...  2019-09-18  2019   
1       ARRÊT DE LA COUR (cinquième chambre)  juille...  2019-07-29  2019   
2       JUDGMENT OF THE COURT (Fifth Chamber)  May (...  2019-05-23  2019   
3       JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
4       JUDGMENT OF THE COURT (First Chamber)  May (...  2019-05-08  2019   
...                                                 ...         ...   ...   
1045    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1046    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1047    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1048    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   
1049    JUDGMENT OF THE GENERAL COURT (Third Chamber...  2010-03-02  2010   

           outcome  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4   

In [40]:
df6.to_csv('Preliminary_database5.csv')